The data provided contains information about individual study results for mathematics, together with demographic information about the student.
Solve each subquest in one cell. Show, if applicable, at the end of each cell first 5 lines of your last result (s).

In [1]:
import pandas as pd

Read the file demographic.csv and show the first 5 lines

In [2]:
demographic = pd.read_csv('https://raw.githubusercontent.com/HOGENT-Databases/DB3-Workshops/master/data/demographic.csv')
demographic.head()

,StudentID,gender,race/ethnicity,parental level of education
0,0,female,group B,bachelor's degree
1,1,female,group C,some college
2,2,female,group B,master's degree
3,3,male,group A,associate's degree
4,4,male,group C,some college


Show the number of rows and columns in demographic.csv

In [3]:
print(demographic.shape)

(1000, 4)


Read the file studentscore.csv and show the first 5 lines

In [4]:
student = pd.read_csv('https://raw.githubusercontent.com/HOGENT-Databases/DB3-Workshops/master/data/studentscore.csv')
student.head()

,StudentID,lunch,test preparation course,math score
0,0,standard,none,72
1,1,standard,completed,69
2,2,standard,none,90
3,3,free/reduced,none,47
4,4,standard,none,76


Show the number of rows and columns in studentscore.csv

In [5]:
print(student.shape)

(1000, 4)


Merge both data frames so that you have one line left per student.

In [6]:
student = pd.merge(student,demographic, on='StudentID', how='inner')
student.head()

,StudentID,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,0,standard,none,72,female,group B,bachelor's degree
1,1,standard,completed,69,female,group C,some college
2,2,standard,none,90,female,group B,master's degree
3,3,free/reduced,none,47,male,group A,associate's degree
4,4,standard,none,76,male,group C,some college


Show the number of male and female students

In [7]:
student.groupby('gender')['StudentID'].count()

gender
female    518
male      482
Name: StudentID, dtype: int64

Show per 'gender' and per 'parental level of education' the average 'math score' in a pivot table. 
Put 'gender' in the columns and 'parental level of education' in the rows.

In [8]:
student.groupby(['parental level of education', 'gender'])['math score'].mean().unstack().fillna(0)

gender,female,male
parental level of education,,
associate's degree,65.250000,70.764151
bachelor's degree,68.349206,70.581818
high school,59.351064,64.705882
master's degree,66.500000,74.826087
some college,65.406780,69.009259
some high school,59.296703,67.840909


Later on we will make a prediction model for the 'math score'. 
Remove the column or columns that are not relevant for this purpose.

In [9]:
student = student.drop(['StudentID'], axis=1)
student.head(5)

,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,standard,none,72,female,group B,bachelor's degree
1,standard,completed,69,female,group C,some college
2,standard,none,90,female,group B,master's degree
3,free/reduced,none,47,male,group A,associate's degree
4,standard,none,76,male,group C,some college


Remove all lines that contain empty fields in some columns.

In [10]:
student = student.dropna()
student.head(5)

,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,standard,none,72,female,group B,bachelor's degree
1,standard,completed,69,female,group C,some college
2,standard,none,90,female,group B,master's degree
3,free/reduced,none,47,male,group A,associate's degree
4,standard,none,76,male,group C,some college


Replace the textual values of 'lunch', 'test preparation course' and 'gender' by numeric values 0 and 1.

In [11]:
import numpy as np

def f(status):
    mapping = {'standard': 0, 'free/reduced': 1}
    return mapping[status.strip()]

student['lunch'] = student['lunch'].map(f)

student['test preparation course'] = np.where(student['test preparation course'] == 'none', 0, 1)

student['gender'] = np.where(student['gender'] == 'male', 0, 1)

student.head()

,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,0,0,72,1,group B,bachelor's degree
1,0,1,69,1,group C,some college
2,0,0,90,1,group B,master's degree
3,1,0,47,0,group A,associate's degree
4,0,0,76,0,group C,some college


Use one-hot-encoding for the columns 'race/ethnicity'
and 'parental level of education'

In [12]:
student = pd.get_dummies(student,columns=['race/ethnicity'], prefix=['re'])
student = pd.get_dummies(student,columns=['parental level of education'], prefix=['edu'])
student.head()

,lunch,test preparation course,math score,gender,re_group A,re_group B,re_group C,re_group D,re_group E,edu_associate's degree,edu_bachelor's degree,edu_high school,edu_master's degree,edu_some college,edu_some high school
0,0,0,72,1,0,1,0,0,0,0,1,0,0,0,0
1,0,1,69,1,0,0,1,0,0,0,0,0,0,1,0
2,0,0,90,1,0,1,0,0,0,0,0,0,1,0,0
3,1,0,47,0,1,0,0,0,0,1,0,0,0,0,0
4,0,0,76,0,0,0,1,0,0,0,0,0,0,1,0


Why is one-hot-encoding preferable over numeric values in this case?

In [13]:
# for instance: there is no "ranking" in race/ethnicity

Create a model to predict the 'math score' based on the other relevant features. Use the RandomForestRegressor method. Automatically search for the optimal number of trees between 50 and 400 in increments of 50. Use RMSE (root mean squared error or the square root of mean squared error) to determine the accuracy of each model. Show the number of trees of the optimal model and the corresponding RMSE.

In [14]:
from sklearn.model_selection import train_test_split
X = student.drop('math score',axis=1)
y = student['math score']

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

X_remainder, X_test, y_remainder, y_test = train_test_split(X,y,test_size=0.30)

best_MAE = 1000
best_trees = 0

for trees in range(50,500,50):
    X_train, X_validation, y_train, y_validation = train_test_split(X_remainder,y_remainder,test_size=0.30)
    model = RandomForestRegressor(n_estimators=trees)

    model.fit(X_train, y_train)    
    y_validation2 = model.predict(X_validation)
    MAE = mean_absolute_error(y_validation, y_validation2)
    if MAE < best_MAE:
        best_MAE = MAE
        best_trees = trees
        best_validation = model.predict(X_test)
        
print('Optimal number of trees = % s' %(best_trees))
print('MAE on validation set = % 3.2f' % (best_MAE)) 
MAEOnTestSet = mean_absolute_error(y_test, best_validation)
print('MAE on test set = % 3.2f' % (MAEOnTestSet))
r2 = r2_score(y_test,best_validation)
print('R square: ' + str(r2))   

Optimal number of trees = 150
MAE on validation set =  11.72
MAE on test set =  11.87
R square: -0.010130626148740829


If you run the program several times, do you always get the same optimal number of trees as best? Explain!

In [15]:
# two kinds of randomization:
# - train_test_split
# - RANDOM forest

Show the relative importance of each of the features arranged according to decreasing importance. However, you have to merge the one-hot-encoded features, adding up their relative importance. Hint: df.reset_index() turns the index into a column. 

In [16]:
imp = pd.DataFrame(model.feature_importances_,columns=['Importance'],index=X_train.columns).sort_values(by='Importance',ascending=False)

In [17]:
imp.index

Index(['lunch', 'test preparation course', 'gender', 're_group E',
       're_group B', 'edu_bachelor's degree', 'edu_some high school',
       'edu_some college', 'edu_associate's degree', 'edu_high school',
       're_group C', 're_group D', 're_group A', 'edu_master's degree'],
      dtype='object')

In [18]:
imp = imp.reset_index()
print(imp)

                      index  Importance
0                     lunch    0.217415
1   test preparation course    0.102342
2                    gender    0.100200
3                re_group E    0.080429
4                re_group B    0.068496
5     edu_bachelor's degree    0.060727
6      edu_some high school    0.057397
7          edu_some college    0.057146
8    edu_associate's degree    0.056057
9           edu_high school    0.054432
10               re_group C    0.047256
11               re_group D    0.045172
12               re_group A    0.028084
13      edu_master's degree    0.024846


In [19]:
imp['index'] = np.where(imp['index'].str.startswith('re_'),'race/ethnicity',imp['index'])
imp['index'] = np.where(imp['index'].str.startswith('edu_'),'parental level of education',imp['index'])
print(imp)

                          index  Importance
0                         lunch    0.217415
1       test preparation course    0.102342
2                        gender    0.100200
3                race/ethnicity    0.080429
4                race/ethnicity    0.068496
5   parental level of education    0.060727
6   parental level of education    0.057397
7   parental level of education    0.057146
8   parental level of education    0.056057
9   parental level of education    0.054432
10               race/ethnicity    0.047256
11               race/ethnicity    0.045172
12               race/ethnicity    0.028084
13  parental level of education    0.024846


In [20]:
# the result of groupby and sum is of datatype Seris, so sort_values doesn't need a by='col' parameter
imp.groupby('index')['Importance'].sum().sort_values(ascending=False)

index
parental level of education    0.310606
race/ethnicity                 0.269438
lunch                          0.217415
test preparation course        0.102342
gender                         0.100200
Name: Importance, dtype: float64